In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     
import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
# include resnet model
batch_size = 128
model_name = "project_resnet20"
model = resnet20_quant()
# model.layer1[0].conv1 = QuantConv2d(8, 8, kernel_size=3, stride=1, padding=1, bias=False)
# model.layer1[0].bn2 = nn.Sequential()

# inspect the model structure
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    adjust_list = [60, 80]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1  


=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
 

In [ ]:
# Training session

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [2]:
# Load the checkpoint

PATH = 'result/'+str(model_name)+'/model_best.pth.tar'
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8799/10000 (88%)



In [3]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)  # Save the input tensor
    def clear(self):
        self.outputs = []  

save_output = SaveOutput()
device = torch.device("cuda" if use_gpu else "cpu") 
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)    

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.cuda()
out = model(images)


7 -th layer prehooked
9 -th layer prehooked
15 -th layer prehooked
17 -th layer prehooked
23 -th layer prehooked
25 -th layer prehooked
32 -th layer prehooked
34 -th layer prehooked
40 -th layer prehooked
44 -th layer prehooked
46 -th layer prehooked
52 -th layer prehooked
54 -th layer prehooked
61 -th layer prehooked
63 -th layer prehooked
69 -th layer prehooked
73 -th layer prehooked
75 -th layer prehooked
81 -th layer prehooked
83 -th layer prehooked


In [4]:
# make a integer version to be able to run on the hardware
w_bit = 4

weight_q = model.layer1[0].conv2.weight_q             # quantized value is stored during the training

w_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha      # alpha is defined in your model already. bring it out here

w_delta = w_alpha / (2**(w_bit-1)-1)                             # delta can be calculated by using alpha and w_bit, for sign number, do minus one for b 
weight_int = weight_q / w_delta    # w_int can be calculated by weight_q and w_delta

print(weight_int) # you should see clean integer numbers

tensor([[[[-1.0000, -5.0000,  3.0000],
          [ 4.0000, -4.0000, -6.0000],
          [ 1.0000, -0.0000,  1.0000]],

         [[-4.0000, -3.0000, -3.0000],
          [ 0.0000, -1.0000,  2.0000],
          [ 4.0000,  3.0000,  2.0000]],

         [[ 5.0000,  1.0000, -2.0000],
          [-1.0000,  4.0000,  0.0000],
          [-1.0000, -2.0000, -2.0000]],

         [[-4.0000, -2.0000, -3.0000],
          [-3.0000,  2.0000,  2.0000],
          [-2.0000, -0.0000, -4.0000]],

         [[-1.0000,  1.0000,  1.0000],
          [ 2.0000,  4.0000,  3.0000],
          [ 0.0000,  1.0000,  4.0000]],

         [[ 6.0000,  1.0000, -3.0000],
          [ 0.0000, -2.0000, -4.0000],
          [ 1.0000, -2.0000,  1.0000]],

         [[-0.0000, -2.0000, -1.0000],
          [ 0.0000,  4.0000,  1.0000],
          [-0.0000,  3.0000,  4.0000]],

         [[ 2.0000,  1.0000, -3.0000],
          [ 4.0000,  3.0000, -3.0000],
          [ 2.0000,  3.0000,  2.0000]]],


        [[[-0.0000,  0.0000,  2.0000],
       

In [5]:
x_bit = 4    
x = save_output.outputs[2][0]  # input of the 2nd conv layer 
x_alpha  = model.layer1[0].conv2.act_alpha
x_delta = x_alpha / (2**x_bit-1)   

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0.0000,  4.0000, 14.0000,  ...,  6.0000,  9.0000,  9.0000],
          [ 3.0000,  7.0000,  9.0000,  ...,  1.0000,  0.0000,  0.0000],
          [ 3.0000,  8.0000,  6.0000,  ...,  4.0000,  1.0000,  0.0000],
          ...,
          [ 1.0000,  3.0000,  3.0000,  ...,  3.0000,  4.0000,  7.0000],
          [ 3.0000,  5.0000,  5.0000,  ...,  5.0000,  5.0000,  7.0000],
          [13.0000, 15.0000, 15.0000,  ..., 15.0000, 15.0000, 14.0000]],

         [[15.0000, 15.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [11.0000, 11.0000,  1.0000,  ...,  0.0000,  0.0000,  1.0000],
          [10.0000,  8.0000,  0.0000,  ...,  1.0000,  0.0000,  5.0000],
          ...,
          [ 8.0000,  7.0000,  8.0000,  ...,  7.0000, 10.0000, 10.0000],
          [ 9.0000, 11.0000, 12.0000,  ..., 11.0000, 12.0000, 10.0000],
          [10.0000, 12.0000, 12.0000,  ..., 12.0000, 14.0000,  6.0000]],

         [[ 4.0000,  6.0000, 13.0000,  ..., 15.0000, 14.0000, 11.0000],
          [ 3.0000,  4.0000,  

In [6]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding =1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = nn.ReLU()
bn = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device)

output_int =  conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_int = bn(output_int)
output_recovered = output_int * x_delta * w_delta # recover with x_delta and w_delta
output_recovered = relu(output_recovered)
print(output_recovered)

tensor([[[[0.0000e+00, 0.0000e+00, 1.5243e-01,  ..., 1.0681e-01,
           1.4243e-01, 1.9305e-01],
          [0.0000e+00, 0.0000e+00, 1.3430e-01,  ..., 6.1938e-03,
           3.7440e-02, 1.2368e-01],
          [0.0000e+00, 0.0000e+00, 1.6867e-01,  ..., 0.0000e+00,
           3.8065e-02, 7.2436e-02],
          ...,
          [0.0000e+00, 0.0000e+00, 3.6941e-03,  ..., 3.6941e-03,
           0.0000e+00, 5.6947e-04],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 1.3045e-02,  ..., 2.0565e-03,
           8.2055e-02, 0.0000e+00],
          [0.0000e+00, 5.4803e-02, 9.8319e-02,  ..., 1.1766e-01,
           2.0601e-01, 2.9309e-02],
          [1.9199e-02, 1.0491e-01, 4.9968e-02,  ..., 5.4803e-02,
           1.3700e-01, 2.5792e-02],
          ...,
          [1.0359e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000

In [7]:
difference = abs( save_output.outputs[3][0] - output_recovered )
print(difference.mean())  ## It should be small < 1 for the trained model

tensor(0.4711, device='cuda:0', grad_fn=<MeanBackward0>)


In [8]:
x_int.size()

torch.Size([128, 8, 32, 32])

In [9]:
x_pad = torch.zeros(8, 34, 34).cuda()
x_pad[ :, 1:33, 1:33] = x_int[0].cuda()
X = x_pad[:, 0:2, 0:10]
X.size()

torch.Size([8, 2, 10])

In [10]:
X = torch.reshape(X, (X.size(0), -1))
X.size()

torch.Size([8, 20])

In [11]:
### storing activation data ###

from pathlib import Path
# Define the folder path
folder_path = Path('./resnet_output/')

# Create the folder if it doesn't exist
folder_path.mkdir(parents=True, exist_ok=True)

bit_precision = 4
file = open('./resnet_output/activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(round(X[7-j,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    


In [12]:
X[:, 0]

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [13]:
weight_int.size()

torch.Size([8, 8, 3, 3])

In [14]:
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
W.size()

torch.Size([8, 8, 9])

In [15]:
### storing weight data ###                     

bit_precision = 4

file = open('./resnet_output/weight.txt', 'w') 
file.write('#col0row7[msb-lsb],col0row6[msb-lsb],....,col0row0[msb-lsb]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lsb],....,col1row0[msb-lsb]#\n')
file.write('#................#\n')

# be careful about negative numbers
for kij in range(9):
    for i in range(W.size(0)):  
        for j in range(W.size(1)):    # row #
            if (W[i, 7-j, kij].item()<0):
                W_bin = '{0:04b}'.format(round(W[i,7-j, kij].item() + 2**bit_precision))        #check again if it works for neg numbers
            else:
                W_bin = '{0:04b}'.format(round(W[i,7-j, kij].item()))
            for k in range(bit_precision):
                file.write(W_bin[k])        
                #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file   

In [16]:
W[0,:,0]

tensor([-1., -4.,  5., -4., -1.,  6., -0.,  2.], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [17]:
output_int.size()

torch.Size([128, 8, 32, 32])

In [18]:
output = output_int[0]
output = torch.reshape(output, (output.size(0), -1))
O = output [:,0:8]
O.size()

torch.Size([8, 8])

In [19]:
### Store output value as well ###

bit_precision = 16
file = open('./resnet_output/output.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lsb],....,time0col0[msb-lsb]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lsb],....,time1col0[msb-lsb]#\n')
file.write('#................#\n')

for i in range(O.size(1)):  # time step
    for j in range(O.size(0)): # array size
        if (O[7-j,i].item()<0):
            O_bin = '{0:016b}'.format(round(O[7-j,i].item() + 2**bit_precision))
        else:
            O_bin = '{0:016b}'.format(round(O[7-j,i].item()))
        for k in range(bit_precision):
            file.write(O_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    